In [39]:
import food2vec

from food2vec.semantic_nutrition import Estimator

estimator = Estimator() 

# Search database for nutrition estimates
match = estimator.natural_search("I ate an apple") 

# Search for embeddings in our database
embedding = estimator.embed('apple')

# See the relationship between embeddings
embedding1 = estimator.embed('cooked rice')
embedding2 = estimator.embed('white rice')
relationship = estimator.cosine(embedding1, embedding2)

------ALERT--------
You're using demonstration nutrition data. This is a random subset (~100 items) from our annotated database.
We do not have permission to distribute our full dataset, and we recommend processing your own.
The USDA provides an excellent start: https://fdc.nal.usda.gov/
You can suppress this warning by setting the 'demo_warning' parameter to False.
-------------------


C:\Users\joeip\anaconda3\lib\site-packages\food2vec\semantic_nutrition.py:237: RuntimeWarning: invalid value encountered in double_scalars
  similarity = np.dot(vA, vB) / (np.linalg.norm(vA) * np.linalg.norm(vB))


In [51]:
import mysql.connector
import pandas as pd
db = mysql.connector.connect(host='localhost',user='root',password='root', database='food_recommendation')

In [52]:
ingredient_count = pd.read_sql(("""

SELECT COUNT(ri.ingredient_id), ingredient_name
FROM food_recommendation.recipe_ingredients ri JOIN food_recommendation.ingredients i
ON ri.ingredient_id = i.ingredient_id
GROUP BY ri.ingredient_id
ORDER BY COUNT(ri.ingredient_id) DESC;

"""), db)

# Filter out ingredient that occurs in less than 10 recipes

In [53]:
ingredient_count.columns = ['count','ingredient_name']
ingredient_count = ingredient_conut[ingredient_conut['count']>10]

In [54]:
ingredient_count

,count,ingredient_name
0,843,salt
1,616,garlic clove
2,583,granulated sugar
3,545,eggs
4,487,olive oil
...,...,...
350,11,0% fat greek yogurt
351,11,coarse grain mustard
352,11,poblano pepper
353,11,unbleached flour


In [56]:
embed_dict = {}

for i in range(len(ingredient_count)):
    try:
        embed_dict[ingredient_count.loc[i, 'ingredient_name']] = estimator.embed(ingredient_count.loc[i, 'ingredient_name'])
    except:
        pass

In [58]:
len(embed_dict)

345

In [67]:
from sklearn.decomposition import PCA
import numpy as np

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [83]:
a = list(embed_dict.values())

In [89]:
pca = PCA()
xpca = pca.fit_transform(a)
pca = pd.DataFrame(x_pca)
pca.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-1.424921,-0.468746,0.263889,-0.111084,-0.742269,-0.429527,0.742770,0.241409,-0.624733,0.250023,...,-9.251449e-17,2.061627e-16,1.757601e-16,-8.645275e-17,-2.174640e-16,-2.544817e-17,-1.446278e-16,1.147021e-16,-6.381841e-17,-1.739311e-16
1,0.712685,2.170885,-0.742725,-0.378764,-0.696280,-0.459939,0.084464,0.268995,-0.140498,-1.455596,...,3.041960e-17,-1.274572e-18,9.111500e-19,-5.263494e-19,-3.969775e-18,3.361608e-18,4.286653e-17,7.813460e-18,3.790046e-19,-1.004088e-17
2,-0.712989,-0.268454,-2.119252,1.413205,-0.633587,-0.466588,0.164784,-0.036308,-0.340553,-0.362480,...,3.041960e-17,-1.274572e-18,9.111500e-19,-5.263494e-19,-3.969775e-18,3.361608e-18,4.286653e-17,7.813460e-18,3.790046e-19,-1.004088e-17
3,-0.956254,-0.634287,1.128917,0.482515,0.097201,-0.262208,-0.379578,-0.165181,-0.314114,-0.237788,...,3.041960e-17,-1.274572e-18,9.111500e-19,-5.263494e-19,-3.969775e-18,3.361608e-18,4.286653e-17,7.813460e-18,3.790046e-19,-1.004088e-17
4,-1.245268,0.632515,-0.165702,0.864141,0.617494,0.321360,0.306528,1.216226,-0.627416,-1.441007,...,-6.609979e-17,-2.065034e-16,2.823645e-16,-1.790635e-16,-6.614154e-17,-1.459488e-16,-6.180621e-17,-7.522828e-17,-3.900998e-16,-1.567552e-16


In [146]:
ingredient = list(embed_dict.keys())
pca.index = ingredient

In [147]:
pca.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
salt,-1.424921,-0.468746,0.263889,-0.111084,-0.742269,-0.429527,0.742770,0.241409,-0.624733,0.250023,...,-9.251449e-17,2.061627e-16,1.757601e-16,-8.645275e-17,-2.174640e-16,-2.544817e-17,-1.446278e-16,1.147021e-16,-6.381841e-17,-1.739311e-16
garlic clove,0.712685,2.170885,-0.742725,-0.378764,-0.696280,-0.459939,0.084464,0.268995,-0.140498,-1.455596,...,3.041960e-17,-1.274572e-18,9.111500e-19,-5.263494e-19,-3.969775e-18,3.361608e-18,4.286653e-17,7.813460e-18,3.790046e-19,-1.004088e-17
granulated sugar,-0.712989,-0.268454,-2.119252,1.413205,-0.633587,-0.466588,0.164784,-0.036308,-0.340553,-0.362480,...,3.041960e-17,-1.274572e-18,9.111500e-19,-5.263494e-19,-3.969775e-18,3.361608e-18,4.286653e-17,7.813460e-18,3.790046e-19,-1.004088e-17
eggs,-0.956254,-0.634287,1.128917,0.482515,0.097201,-0.262208,-0.379578,-0.165181,-0.314114,-0.237788,...,3.041960e-17,-1.274572e-18,9.111500e-19,-5.263494e-19,-3.969775e-18,3.361608e-18,4.286653e-17,7.813460e-18,3.790046e-19,-1.004088e-17
olive oil,-1.245268,0.632515,-0.165702,0.864141,0.617494,0.321360,0.306528,1.216226,-0.627416,-1.441007,...,-6.609979e-17,-2.065034e-16,2.823645e-16,-1.790635e-16,-6.614154e-17,-1.459488e-16,-6.180621e-17,-7.522828e-17,-3.900998e-16,-1.567552e-16


In [101]:
ingredient

['salt',
 'garlic clove',
 'granulated sugar',
 'eggs',
 'olive oil',
 'butter',
 'flour',
 'pepper',
 'water',
 'onion',
 'vanilla',
 'lemon juice',
 'milk',
 'salt and pepper',
 'brown sugar',
 'unsalted butter',
 'kosher salt',
 'baking powder',
 'fresh parsley',
 'parmesan cheese',
 'baking soda',
 'scallions',
 'heavy cream',
 'canola oil',
 'honey',
 'carrots',
 'cilantro',
 'tomato',
 'ground cinnamon',
 'vegetable oil',
 'powdered sugar',
 'fresh ginger root',
 'lime juice',
 'ground cumin',
 'garlic powder',
 'oregano',
 'basil',
 'cream cheese',
 'red bell peppers',
 'cornstarch',
 'chile powder',
 'red onion',
 'lemon zest',
 'sour cream',
 'red pepper flakes',
 'bell pepper',
 'cinnamon',
 'fresh thyme leaves',
 'walnuts',
 'canned tomatoes',
 'shallots',
 'egg white',
 'soy sauce',
 'cocoa powder',
 'dijon mustard',
 'fresh mushrooms',
 'pecans',
 'extra-virgin olive oil',
 'celery',
 'mayo',
 'cayenne pepper',
 'white wine',
 'egg yolk',
 'bacon',
 'avocado',
 'potato',
 

In [104]:
import plotly.graph_objects as go

In [143]:
s = {}

for i in embed_dict.keys():
    try:
        s[i] = ingredient_count[ingredient_count['ingredient_name'] == i]['count'].iloc[0]
    except:
        pass

In [190]:
size = []
for i in ingredient:
    size.append(s[i])

size = np.log(size)

In [203]:
size2 =  size * 2

In [204]:
node_x = pca[0]
node_y = pca[1]

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text = ingredient,
    textfont = {'size':size2},
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Greens',
        reversescale=False,
        color=[],
        size=1,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

In [205]:
# plot diagram
fig = go.Figure(data=[node_trace],
             layout=go.Layout(
                width = 1000,
                height = 1000,
                title='PCA of food embedding',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(

                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)),
                
                )



fig.show()